# Read and save training outputs in a handy format

In [ ]:
# python libraries
import glob
import numpy as np
import h5py
import math
import time
import matplotlib.pyplot as plt
import keras.backend as K
from keras.models import model_from_json
import tensorflow as tf
from scipy.stats import norm, expon, chi2
from scipy.stats import chisquare
import os
import getpass

# open access to cernbox (password cern account)
os.system("echo %s| kinit" %getpass.getpass())

In [ ]:
def collect_t(DIR_IN, DIR_OUT):
    '''
    For each toy the function reads the .txt file where the final value for the variable t=-2*loss is saved. 
    It then associates a label to each toy.
    The array of the t values (tvalues) and the array of labels (files_id) are saved in an .h5 file.
    
    DIR_IN: directory where all the toys' outputs are saved
    DIR_OUT: directory where to save the .h5 output file
    
    The function returns the array of labels.
    '''
    tvalues = np.array([])
    files_id = np.array([])
    FILE_TITLE=''
    for fileIN in glob.glob("%s/*_t.txt" %DIR_IN):
        #print(fileIN)
        f = open(fileIN)
        lines = f.readlines()
        file_id=  fileIN.split('/')[-1]
        FILE_TITLE = fileIN.split('/')[-2]
        file_id = file_id.replace('_t.txt', '')
        #print(file_id)
        if len(lines)==0:
            continue
        t = float(lines[0])
        #print(file_id)
        if(np.isnan(np.array([t]))): 
            continue    
        tvalues = np.append(tvalues, t)
        files_id = np.append(files_id, file_id)
        
    # save tvalues in a h5 file
    f = h5py.File(DIR_OUT+ FILE_TITLE+'_tvalues.h5', 'w')
    f.create_dataset('tvalues', data=tvalues, compression='gzip')
    f.create_dataset('files_id', data=files_id, compression='gzip')
    f.close()
    
    return files_id

def collect_history(files_id, DIR_IN, patience):
    '''
    For each toy whose file ID is in the array files_id, 
    the function collects the history of the loss and saves t=-2*loss at the check points.
    
    files_id: array of toy labels 
    DIR_IN: directory where all the toys' outputs are saved
    patience: interval between two check points (epochs)
    
    The function returns a 2D-array with final shape (nr toys, nr check points).
    '''
    tdistributions_check =np.array([])
    cnt=0
    for file_id in files_id:
        history_file = DIR_IN+file_id+'_history'+str(patience)+'.h5'
        #print(history_file)
        f = h5py.File(history_file)
        #print(history_file)
        loss = f.get("loss")
        if not loss:
            print("not")
            continue
        loss = np.array(loss)
        loss = np.expand_dims(loss, axis=1)
        if not cnt:
            # initialize the array at the first iteration
            tdistributions_check = -2*loss
        else:
            # just append to tdistributions_check
            tdistributions_check = np.concatenate((tdistributions_check, -2*loss), axis=1)
        print(str(cnt)+': toy '+file_id+' loaded.')
        cnt = cnt+1
        #print(tdistributions_check.shape)
    print('Final history array shape')
    print('(nr toys, nr check points)')
    print(tdistributions_check.T.shape)
    return tdistributions_check.T

    

def Save_to_h5(DIR_OUT, file_name, extension, patience, tvalues_check):
    '''
    The function save the 2D-array of the loss histories in an .h5 file.
    
    DIR_OUT: directory where to save the output file
    file_name: output file name
    extension: label to be appended to the file_name
    
    No return.
    '''
    epochs_check = []
    nr_check_points = tvalues_check.shape[1]
    for i in range(nr_check_points):
        epoch_check = patience*(i+1)
        epochs_check.append(epoch_check)
        
    log_file = DIR_OUT+file_name+extension+'.h5' #'_tvalues_check.h5'
    print(log_file)
    f = h5py.File(log_file,"w")
    for i in range(tvalues_check.shape[1]):
        f.create_dataset(str(epochs_check[i]), data=tvalues_check[:, i], compression='gzip')
    f.close()
    print('Saved to file: ' +file_name+extension+'.h5')
    return

In [ ]:
# output directory
output_path='./....'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# input directory
DIR_INPUT = './....'

# 
patience = DIR_INPUT.split("patience",1)[1] 
patience = patience.split("_",1)[0]

In [ ]:
if not DIR_INPUT.endswith('/'):
    DIR_INPUT=DIR_INPUT+'/'
    
title = DIR_INPUT.split('/')[-2]
print(title)

files_id = collect_t(DIR_INPUT, output_path)
print('Loaded') 
tvalues_check = collect_history(files_id, DIR_INPUT, int(patience))

Save_to_h5(output_path, title, '_tvalues_check', int(patience), tvalues_check)

# ANALYSIS

## functions definition

In [ ]:
def Read_history_from_h5(DIR_IN, file_name, extension, patience, epochs):
    '''
    The function creates a 2D-array from a .h5 file.
    
    DIR_OUT: directory where to save the input file
    file_name: input file name
    extension: label to be appended to the file_name
    
    The function returns a 2D-array with final shape (nr toys, nr check points). 
    '''
    
    tvalues_check = np.array([])
    epochs_check = []
    
    for i in range(epochs/patience):
        epoch_check = patience*(i+1)
        epochs_check.append(epoch_check)
        
    log_file = DIR_IN+file_name+extension+'.h5'
    print(log_file)

    f = h5py.File(log_file,"r")
    for i in range(len(epochs_check)):
        # the t distribution at each check point is named by the epoch number
        t = f.get(str(epochs_check[i]))
        t = np.array(t)
        t = np.expand_dims(t, axis=1)
        #print(t.shape)
        if not i:
            # initialize the array at the first iteration
            tvalues_check = t
        else:
            # just append to tvalues_check
            tvalues_check = np.concatenate((tvalues_check, t), axis=1)
    f.close()
    print(tvalues_check.shape)
    return tvalues_check

def Read_t_from_h5(DIR_IN, file_name, extension):
    log_file = DIR_IN+file_name+extension+'.h5'
    print(log_file)
    tvalues_check = np.array([])
    f = h5py.File(log_file,"r")
    t = f.get('tvalues')
    t = np.array(t)
    print(t.shape)
    return t

In [ ]:
def Plot_Percentiles_v1(tvalues_check, patience, title='', ymax=300, ymin=0):
    '''
    The funcion creates the plot of the evolution in the epochs of the [2.5%, 25%, 50%, 75%, 97.5%] quantiles of the toy sample distribution.
    
    patience: interval between two check points (epochs).
    tvalues_check: t=-2*loss
    '''
    epochs_check = []
    nr_check_points = tvalues_check.shape[1]
    for i in range(nr_check_points):
        epoch_check = patience*(i+1)
        epochs_check.append(epoch_check)
    
    fig=plt.figure(figsize=(8, 8))
    quantiles=[2.5, 25, 50, 75, 97.5]
    percentiles=np.array([])
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('t', fontsize=12)
    plt.ylim(ymin, ymax)
    for i in range(tvalues_check.shape[1]):
        percentiles_i = np.percentile(tvalues_check[:, i], quantiles)
        #print(percentiles_i.shape)
        percentiles_i = np.expand_dims(percentiles_i, axis=1)
        #print(percentiles_i.shape)
        if not i:
            percentiles = percentiles_i.T
        else:
            percentiles = np.concatenate((percentiles, percentiles_i.T))
    legend=[]
    print(percentiles.shape)
    for j in range(percentiles.shape[1]):
        plt.plot(epochs_check, percentiles[:, j], marker='.')
        legend.append(str(quantiles[j])+' % quantile')
    plt.legend(legend, fontsize=13)
    plt.grid()
    plt.show()
    fig.savefig(output_path+title+'_PlotPercentiles.png')
    plt.close(fig)
    return

def Plot_Percentiles_v2(tvalues_check, patience, title, dof, wc, ymax=300, ymin=0):
    '''
    The funcion creates the plot of the evolution in the epochs of the [2.5%, 25%, 50%, 75%, 97.5%] quantiles of the toy sample distribution.
    
    patience: interval between two check points (epochs).
    tvalues_check: t=-2*loss
    '''
    epochs_check = []
    nr_check_points = tvalues_check.shape[1]
    for i in range(nr_check_points):
        epoch_check = patience*(i+1)
        epochs_check.append(epoch_check)
    
    fig=plt.figure(figsize=(8, 8))
    quantiles=[2.5, 25, 50, 75, 97.5]
    percentiles=np.array([])
    plt.xlabel('Training Epochs', fontsize=16)
    plt.ylabel('t', fontsize=16)
    plt.ylim(ymin, ymax)
    plt.title('Weight Clipping = '+str(wc), fontsize=16)
    for i in range(tvalues_check.shape[1]):
        percentiles_i = np.percentile(tvalues_check[:, i], quantiles)
        #print(percentiles_i.shape)
        percentiles_i = np.expand_dims(percentiles_i, axis=1)
        #print(percentiles_i.shape)
        if not i:
            percentiles = percentiles_i.T
        else:
            percentiles = np.concatenate((percentiles, percentiles_i.T))
    legend=[]
    #print(percentiles.shape)
    for j in range(percentiles.shape[1]):
        plt.plot(epochs_check, percentiles[:, j], marker='.', linewidth=3)
        #print(percentiles[:, j])
        legend.append(str(quantiles[j])+' % quantile')
    for j in range(percentiles.shape[1]):
        plt.plot(epochs_check, chi2.ppf(quantiles[j]/100., df=dof, loc=0, scale=1)*np.ones_like(epochs_check),
                color='grey', ls='--', linewidth=1)
        #print( chi2.ppf(quantiles[j]/100., df=dof, loc=0, scale=1))
        if j==0:
            legend.append("Target "+r"$\chi^2(dof=$"+str(dof)+")")
            
    plt.legend(legend, fontsize=16)
    #plt.grid()
    plt.show()
    #fig.savefig(output_path+title+'_PlotPercentiles.png')
    plt.close(fig)
    return

In [ ]:
def Plot_CHI2_tests(output_path, titleID, tdistributions_check, patience, 
                    xmin, xmax, bins, dof, 
                    shift=0, plot_patience=50,
                    verbose=0, save=0):
    '''
    The function creates an array of plots for the distribution of t at different check points during the training.
    
    patience: interval between two check points (epochs).
    tdistributions_check: 2Darray of the t distribution history.
    '''
    epochs_check = []
    nr_check_points = tdistributions_check.shape[1]
    print(nr_check_points)
    print(patience)
    for i in range(nr_check_points):
        epoch_check = patience*(i+1)
        epochs_check.append(epoch_check)
    
    # build the reference binned chi2 distribution starting from the cdf
    chi_range = xmax-xmin
    chi_bin = chi_range*1./bins
    chi_ref = np.array([])
    chi_xs = np.array([])
    
    for i in range(bins):
        chi_x = xmin+0.5*(chi_bin*(i)+chi_bin*(i+1))+shift
        chi_xs = np.append(chi_xs, chi_x)
        chi_integral = (chi2.cdf(xmin+chi_bin*(i+1), dof)-chi2.cdf(xmin+chi_bin*(i), dof))
        chi_ref= np.append(chi_ref, chi_integral)
        
    # setting the dimension of the plot grid    
    nplots = int(len(epochs_check)/plot_patience)+(len(epochs_check)%plot_patience > 0)
    ncols = 4
    nrows = int(nplots/4)+ (nplots% 4 > 0)
    fig_height = nrows*7
    fig_width = ncols*5
    fig = plt.figure(figsize=(fig_width, fig_height))
    
    # building histograms
    chi2_tests=[]
    p_val_chi2_tests=[]
    epoch_labels = []
    for check in range(len(epochs_check)):
        if check%plot_patience:
            continue
        plt.subplot(nrows, 4, check/plot_patience+1)
        epoch = epochs_check[check]
        epoch_labels.append(int(epoch/1000.))
        
        t = plt.hist(tdistributions_check[:, check], bins=bins, 
                     alpha=0.7, range=(xmin+shift, xmax+shift), edgecolor='blue')
        chi_ref_lineplot = plt.errorbar(chi_xs, chi_ref*np.sum(t[0]), yerr=np.sqrt(chi_ref*np.sum(t[0])), 
                                 color='darkgreen')
        plt.legend(['t distribution at '+str(epoch_labels[-1])+'k epochs', 'chi2 with '+str(dof)+' dof'],
                   loc='upper center', bbox_to_anchor=(0.5, -0.1)) 
        
        # building a chi2 test of compatibility between histogram and reference distribution
        chi2_obs = t[0]
        chi2_true = chi_ref*np.sum(t[0])
        
        numerator = np.multiply(np.subtract(chi2_obs, chi2_true), np.subtract(chi2_obs, chi2_true))
        denominator = chi2_true
        
        chi2_test = np.sum(np.divide(numerator, denominator))
        chi2_tests.append(chi2_test)
        
        p_val_chi2_test = 1.-chi2.cdf(chi2_test, bins-1)
        p_val_chi2_tests.append(p_val_chi2_test)
        props = dict(boxstyle='square', facecolor='white', alpha=0.1)
        ylmin, ylmax = plt.ylim()
        xlmin, xlmax = plt.xlim()
        textstr = "median: %f" %(np.median(tdistributions_check[:, check].astype(float)))
        
        plt.annotate(textstr, xy=(0.2, 0.9), xycoords='axes fraction',
                 #verticalalignment='top',horizontalalignment='right', 
                 fontsize=11)
    plt.subplots_adjust(left=0.25, bottom = 0.5, wspace=0.3, hspace=0.5)
    if verbose:
        plt.show()
    fig.savefig(output_path+titleID+'_hist_CHI2_dof'+str(dof)+'_'+str(xmin)+'-'+str(xmax)+'bins'+str(bins)+'.png')
    plt.close(fig)
    
    # plot compatibility test results
    fig = plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epoch_labels, chi2_tests)
    plt.plot([0, epoch_labels[-1]],[bins-1, bins-1])
    plt.plot([0, epoch_labels[-1]],[bins-1+np.sqrt(2*(bins-1)), bins-1+np.sqrt(bins-1)])
    plt.plot([0, epoch_labels[-1]],[bins-1+2*np.sqrt(2*(bins-1)), bins-1+2*np.sqrt(bins-1)])
    plt.plot([0, epoch_labels[-1]],[bins-1+3*np.sqrt(2*(bins-1)), bins-1+3*np.sqrt(bins-1)])
    plt.yscale('log')
    plt.xlabel('Epochs [k]')
    plt.ylabel('chi2 test')
    plt.grid()
    plt.legend(['chi2 obs','dof '+str(bins-1), 'dof + 1 sigma', 'dof + 2 sigma', 'dof + 3 sigma'])
    plt.subplot(1, 2, 2)
    plt.plot(epoch_labels, p_val_chi2_tests)
    plt.plot([0, epoch_labels[-1]],[0.05, 0.05])
    plt.plot([0, epoch_labels[-1]],[0.32, 0.32])
    plt.plot([0, epoch_labels[-1]],[0.5, 0.5])
    plt.legend(['p-values', '0.05', '0.32', '0.5'])
    plt.ylabel('p-value')
    plt.xlabel('Epochs [k]')
    #plt.yscale('log')
    print(epoch_labels)
    print(chi2_tests)
    print(p_val_chi2_tests)
    #save into .h5
    log_file=output_path+'/'+titleID+'_CHI2_test.h5'
    f = h5py.File(log_file,"w")
    f.create_dataset(titleID.split('_')[-1], data=chi2_tests, compression='gzip')
    f.create_dataset('epochs', data=epoch_labels, compression='gzip')
    f.close()
    plt.grid()
    #plt.subplots_adjust(left=0.25, bottom = 0.3, wspace=0.3, hspace=0.3)
    if verbose:
        #plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.3, wspace=0.4)
        plt.subplots_adjust(left=0.125, right=0.9, bottom = 0.25, wspace=0.3, hspace=0.2)
        plt.show()
    if save:
        fig.savefig(output_path+titleID+'_plot_CHI2_dof'+str(dof)+'_'+str(xmin)+'-'+str(xmax)+'bins'+str(bins)+'.png')
    plt.close(fig) 
    print(np.max(p_val_chi2_tests))
    return

In [ ]:
def Plot_Analysis_tdistribution(output_path, title, tvalues_BkgOnly, tvalues, dof, rmin, rmax, bins=35, verbose=0, save=0):
    '''
    The function creates the plot for the comparison of two samples of toys at the end of the training.
    tvalues_BkgOnly: t distribution for the sample with BKG-only events.
    tvalues: t distribution for the sample with Sig+Bkg events.
    dof: number of degrees of freedom of the reference chi2 distribution.
    
    '''
    fig, ax = plt.subplots()
    chisq = np.random.chisquare(dof, 5000000)
    ax.hist(chisq, bins=bins, range=(rmin, rmax), density = True, histtype = 'step', linewidth=2, color='darkgreen')
    ax.hist(tvalues_BkgOnly, bins=bins, range=(rmin, rmax), density = True, alpha = 0.7, edgecolor='blue')
    ax.hist(tvalues, bins=bins, range=(rmin, rmax), density= True, alpha = 0.7, edgecolor='red')
    ax.legend(["$\chi^2$ with "+str(dof)+" dof",'Data samples following SM','Data samples containing New Physics'], loc='upper right')
    ax.set_ylabel('Probability')
    ax.set_xlabel("t")
    ax.set_title(title)
    #compute significance
    #median = np.median(tvalues)
    quantiles=np.percentile(tvalues, [16., 50., 84.])
    q50=quantiles[1]
    q16=quantiles[0]
    q84=quantiles[2]
    counts50 = np.sum((tvalues_BkgOnly > q50).astype(int))
    counts16 = np.sum((tvalues_BkgOnly > q16).astype(int))
    counts84 = np.sum((tvalues_BkgOnly > q84).astype(int))
    
    p_val50 = counts50*1./len(tvalues_BkgOnly)
    p_val16 = counts16*1./len(tvalues_BkgOnly)
    p_val84 = counts84*1./len(tvalues_BkgOnly)
    
    chisq = np.random.chisquare(dof, 100000000)
    integral50 = (chisq > q50).sum()/float(len(chisq))
    integral16 = (chisq > q16).sum()/float(len(chisq))
    integral84 = (chisq > q84).sum()/float(len(chisq))
    
    print("Bkg-only median: %f" %np.median(tvalues_BkgOnly))
    print("Bkg-only mean: %f" %np.mean(tvalues_BkgOnly))
    print("Bkg-only RMS: %f" %math.sqrt(np.var(tvalues_BkgOnly)))
    print("Sig+Bkg median: %f" %np.median(tvalues))
    print("Sig+Bkg quantile16: %f" %q16)
    print("Sig+Bkg quantile84: %f" %q84)
    print("Sig+Bkg mean: %f" %np.mean(tvalues))
    print("Sig+Bkg RMS: %f" %math.sqrt(np.var(tvalues)))
    
    print("p-value %f with 68 %% CL [%f, %f]" %(p_val50, p_val16, p_val84))
    print("number of sigmas: %f with 68%% CL [%f, %f]" %(norm.ppf(1.-p_val50), norm.ppf(1.-p_val16), norm.ppf(1.-p_val84)))
    print("p-value assuming %i dof chi square: %f" %(dof, integral50))
    print("number of sigmas assuming %i dof chi square: %f with 68 %% CL [%f, %f]" %(dof, norm.ppf(1.-integral50), norm.ppf(1.-integral16), norm.ppf(1.-integral84)))
    textstr = "Bkg-only median: %f\nSig+Bkg median: %f\nSignificance: %f $\sigma$\nTh Significance: %f $\sigma$" %(np.median(tvalues_BkgOnly), np.median(tvalues), norm.ppf(1.-p_val50), norm.ppf(1.-integral50))

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='square', facecolor='white', alpha=0.1)

    # place a text box in upper left in axes coords
    ax.text(0.5, 0.65, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)
    if verbose:
        plt.show()
    fig.savefig(output_path+title+'_t_distributions_dof'+str(dof)+'.png')
    plt.close(fig)
    return 

## Plots

In [ ]:
# where to save plots
output_path='/...'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# where you saved *tvalues_check.h5 and *tvalues.h5
read_path_B='/...'
read_path_SB='/....'


# Collect training parameters from folder name

# where you saved the training outputs
DIR_INPUT_B = '..../Z_5D_patience10000_ref1000000_bkg200000_sig0_epochs600000_latent13_layers1_wclip0.7/'
DIR_INPUT_SB ='..../Z_5D_ref1000000_bkg200000_sig100_epochs300000_latent5_layers3_wclip2.15/'

patience_B = DIR_INPUT_B.split("patience",1)[1] 
patience_B = patience_B.split("_",1)[0]
patience_SB = DIR_INPUT_SB.split("patience",1)[1] 
patience_SB = patience_SB.split("_",1)[0]

wclip_B = DIR_INPUT_B.split("wclip",1)[1] 
wclip_B = wclip_B.split("_",1)[0]
wclip_B = wclip_B.split("/",1)[0]
wclip_SB = DIR_INPUT_SB.split("wclip",1)[1] 
wclip_SB = wclip_SB.split("_",1)[0]
wclip_SB = wclip_SB.split("/",1)[0]

epochs_B = DIR_INPUT_B.split("epochs",1)[1] 
epochs_B = epochs_B.split("_",1)[0]
epochs_SB = DIR_INPUT_SB.split("epochs",1)[1] 
epochs_SB = epochs_SB.split("_",1)[0]

if not DIR_INPUT_B.endswith('/'):
    DIR_INPUT_B = DIR_INPUT_B+'/'
if not DIR_INPUT_SB.endswith('/'):
    DIR_INPUT_SB = DIR_INPUT_SB+'/'

title_SB = DIR_INPUT_SB.split('/')[-2]
title_B = DIR_INPUT_B.split('/')[-2]

In [ ]:
# loading loss history
tvalues_check_B = Read_history_from_h5(read_path_B, title_B, '_tvalues_check', int(patience_B), int(epochs_B))
tvalues_check_SB = Read_history_from_h5(read_path_SB, title_SB, '_tvalues_check', int(patience_SB), int(epochs_SB))

In [ ]:
# loading of the final t distributions
t_B = Read_t_from_h5(read_path_B, title_B, extension='_tvalues')
t_SB = Read_t_from_h5(read_path_SB, title_SB, extension='_tvalues')

### percentile plot

In [ ]:
Plot_Percentiles_v2(tvalues_check_B, int(patience_B), title_B, dof=10, wc=float(wclip_B), ymax=40, ymin=0)

### CHI2 test of the compatibility

In [ ]:
Plot_CHI2_tests(output_path, title_B, tvalues_check_B, 
                patience=10000,
                xmin=10, xmax=70, bins=7,
                dof=40, shift=0, 
                plot_patience=1, verbose=1, save=1)

### test statistic on BSM events

In [ ]:
Plot_Analysis_tdistribution(output_path, title_SB, t_B, t_SB, dof=96, rmin=50, rmax=667, bins=15, verbose=1, save=1)